### Load data

Choose that dataset to load. We have prepared 2 different datasets, downloaded and converted from the competition:
* wbg-fal10: dataset_01 ([ITC2019 description](https://www.itc2019.org/instances/status/5b59b12884ed6a262b1ce9d3))
* pu-cs-fal07: dataset_04 ([ITC2019 description](https://www.itc2019.org/instances/status/5b59ebf984ed6a262b1ce9dc))

Choose between datasets by adjusting the command `import dataset_0x as dataset`

In [1]:
from docplex.cp.model import CpoModel
import time
import dataset_01 as dataset

rooms = dataset.rooms
courses = dataset.courses
students = dataset.students
constraints = dataset.constraints

List of supporting tables:
* `classes`: 150 classes with details
* `classes_id`: ids from 1 to 150
* `rooms_id`: ids from 1 to 7
* `students_id`: ids from 1 to 19
* `student_subparts`: pairs of 1 student & 1 subpart that he/she must attend (the subpart is a list of classes_id)
* `student_classes`: pairs of 1 student & the list of all classes that he/she might attend (flattened from above subpart) -> 19 pairs

In [2]:
rooms_id = [r['id'] for r in rooms]
students_id = [s['id'] for s in students]

# class-time pairs
classes = []
for course in courses:
    for subpart in course['subpart']:
        for c in subpart['class']:
            c['sbp_id'] = subpart['id']
            classes.append(c)
classes_id = [c['id'] for c in classes]

# student-subpart pairs
student_subparts = []
for s in students_id:
    for i in students[s-1]['course']:
        for subpart in courses[i-1]['subpart']:
            student_subparts.append([s,[c['id'] for c in subpart['class']]]) 

# student-classes pairs
student_classes = []
for s in students_id:
    student_classes.append([s, []])
    class_list = student_classes[-1][1]
    for i in students[s-1]['course']:
        for subpart in courses[i-1]['subpart']:
            for c in subpart['class']:
                class_list.append(c['id'])

# pairs of classes with SameAttendee constraint
SA_pairs=[]
for ct in constraints:
    if ct['type']=='SameAttendees':
        for i1 in range(len(ct['class'])):
            for i2 in range(i1+1, len(ct['class'])):
                c1, c2 = ct['class'][i1], ct['class'][i2]
                SA_pairs.append((c1,c2))

### support functions:

In [3]:
def clash(time1, time2):
    clash_days = bool(sum([int(a) & int(b) for a,b in zip(time1['days'], time2['days'])]))
    clash_weeks = bool(sum([int(a) & int(b) for a,b in zip(time1['weeks'], time2['weeks'])]))
    clash_time1 = time1['start'] < time2['start'] + time2['length']
    clash_time2 = time1['start'] + time1['length'] > time2['start']
    return clash_days and clash_weeks and clash_time1 and clash_time2

# Modeling

### Decision Variables

In [4]:
m = CpoModel('model1')

x = m.integer_var_list(len(classes_id), name='x')
y = m.integer_var_list(len(classes_id), name='y')
z = m.integer_var_dict([(s,sbp['id']) for s in students_id 
                        for i in students[s-1]['course'] 
                        for sbp in courses[i-1]['subpart']], 
                       name='z')

for c in classes_id:
    x[c-1].set_domain([i for i in range(0, len(classes[c-1]['time']))])
    y[c-1].set_domain([i for i in range(0, len(classes[c-1]['room']))])

for s in students_id:
    for i in students[s-1]['course']:
        for sbp in courses[i-1]['subpart']:
            z[s,sbp['id']].set_domain([c['id'] for c in sbp['class']])

* penalty variables are declared to serve the purpose of setting up objective only (they are directly derived from x,y variables)

In [5]:
# penalty variables for computing objective
pr = m.integer_var_list(len(classes_id), name='pr')
pt = m.integer_var_list(len(classes_id), name='pt')
for c in classes_id:
    pr[c-1].set_domain(set([croom['penalty'] for croom in classes[c-1]['room']]))
    pt[c-1].set_domain(set([ctime['penalty'] for ctime in classes[c-1]['time']]))

### Objective functions
(skip if only search for feasible solution)

In [6]:
room_penalty = m.sum(pr)
time_penalty = m.sum(pt)
m.minimize(room_penalty + time_penalty)

### Constraints

In [7]:
# (Implied within CP variables)
# Every class must be assigned a time
# Every class must be assigned a room, where applicable
# Every student must attend exactly one class for each subpart that he/she must attend   

# Constraint to define penalty variables
for c in classes_id:
    for r in range(len(classes[c-1]['room'])):
        m.add(m.if_then(y[c-1] == r, pr[c-1] == classes[c-1]['room'][r]['penalty']))
        
for c in classes_id:
    for t in range(len(classes[c-1]['time'])):
        m.add(m.if_then(x[c-1] == t, pt[c-1] == classes[c-1]['time'][t]['penalty']))

# For two classes with a parent-child relationship, if a class is assigned to a student
# then the parent class must also be assigned
for s, subpart in student_subparts:
    for c in subpart:
        if classes[c-1].get('parent', 0):
            parent = classes[c-1]['parent']
            m.add(m.if_then(z[s,classes[c-1]['sbp_id']] == c, 
                            z[s, classes[parent-1]['sbp_id']] == parent))

# The capacity of each class in terms of the number of students must be satisfed
for c in classes_id:
    m.add(m.count([z[i] for i in z], c) <= classes[c-1]['limit'])
    
# A room cannot be used when it is unavailable
# there are no clashes in this test set
for c in classes_id:
    for t in range(len(classes[c-1]['time'])):
        for r in range(len(classes[c-1]['room'])):
            ctime = classes[c-1]['time'][t]
            room = rooms[classes[c-1]['room'][r]['id'] - 1]
            penalty = classes[c-1]['room'][r]['penalty']
            for room_uat in room['unavailable']:
                if clash(ctime, room_uat):
                    m.add(m.if_then(x[c-1] == t, y[c-1] != r))
                    
# Any hard distribution constraints must be satisfied
# in this instance refers to the same attendee requirement
for c1, c2 in SA_pairs:
    for t1 in range(len(classes[c1-1]['time'])):
        for t2 in range(len(classes[c2-1]['time'])):
            time1=classes[c1-1]['time'][t1]
            time2=classes[c2-1]['time'][t2]
            if clash(time1, time2):
                m.add(m.if_then(x[c1-1] == t1, x[c2-1] != t2))

* Two class cannot be at the same time and in the same room (H8) (~3 minutes)

In [8]:
# Two class cannot be at the same time and in the same room
# Long time to run
for i1 in range(len(classes_id)):
    for i2 in range(i1+1, len(classes_id)):
        c1, c2 = classes_id[i1], classes_id[i2]
            
        for t1 in range(len(classes[c1-1]['time'])):
            for t2 in range(len(classes[c2-1]['time'])):
                time1=classes[c1-1]['time'][t1]
                time2=classes[c2-1]['time'][t2]
                if clash(time1, time2):

                    for r1 in range(len(classes[c1-1]['room'])):
                        for r2 in range(len(classes[c2-1]['room'])):
                            r1_id=classes[c1-1]['room'][r1]['id']
                            r2_id=classes[c2-1]['room'][r2]['id']
                            if r1_id==r2_id:
                                m.add(m.if_then(m.logical_and(m.logical_and(x[c1-1]==t1, x[c2-1]==t2), y[c1-1]==r1), 
                                                y[c2-1]!=r2))

In [9]:
print(m.get_statistics())

IntegerVars: 799, IntervalVars: 0, Constraints: 606301, Exprs: 606301, Nodes: 6579992, Ops: 9


### support functions:

In [10]:
def check_solution(sol):
    print('---no output if all constraints passed---')
    
    # (4) check parent-child relationship
    for s in students_id:
        for i in students[s-1]['course']:
            sbp_id = courses[i-1]['subpart'][0]['id']
            ctaken = sol[z[s, sbp_id]]
            if classes[ctaken-1].get('parent', 0):
                cparent = classes[ctaken-1]['parent']
                sbp_parent = classes[cparent-1]['sbp_id']
                if z[s, sbp_parent] != cparent:
                    print('Constraint (4) violated!')
                    break

    # (5) check class limit
    for c in classes_id:
        if len([i for i in z if sol[z[i]]==c]) > classes[c-1]['limit']:
            print('Constraint (5) violated!')
            break
        
    # (6) check room unavailable time
    for c in classes_id:
        ctime = classes[c-1]['time'][sol[x[c-1]]]
        room_id = classes[c-1]['room'][sol[y[c-1]]]['id']
        for room_uat in rooms[room_id-1]['unavailable']:
            if clash(ctime, room_uat):
                print('Constraint (6) violated!')
                break
            
    # (7) check SameAttendees
    for c1, c2 in SA_pairs:
        time1 = classes[c1-1]['time'][sol[x[c1-1]]]
        time2 = classes[c2-1]['time'][sol[x[c2-1]]]
        if clash(time1, time2):
            print('Constraint (7) violated!')
            break                          

    # (8) check if 2 classes clash room-time constraint
    for i1 in range(len(classes_id)):
        for i2 in range(i1+1, len(classes_id)):
            c1, c2 = classes_id[i1], classes_id[i2]
            time1 = classes[c1-1]['time'][sol[x[c1-1]]]
            time2 = classes[c2-1]['time'][sol[x[c2-1]]]
            room1 = classes[c1-1]['room'][sol[y[c1-1]]]['id']
            room2 = classes[c2-1]['room'][sol[y[c2-1]]]['id']
            # print out details if there is a constraint violation:
            if clash(time1, time2) and room1==room2:
                print('Constraint (8) violated!')
                break

### Solver

In [12]:
sol = m.solve(log_output = True, TimeLimit=1000)

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Minimization problem - 799 variables, 606300 constraints
 ! Presolve      : 139 extractables eliminated, 1 constraint generated
 ! TimeLimit            = 1000
 ! Initial process time : 17.02s (16.98s extraction + 0.04s propagation)
 !  . Log search space  : 1452.8 (before), 1452.8 (after)
 !  . Memory usage      : 242.4 MB (before), 242.4 MB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        799                 -
 + New bound is 0
                     1000        130    1   F     3 != x_97
                     1000        135    2   F     1  = x_79
                     1000         62    3        16 != x_130
                     1000         35    4         1  = x_82
                     1000         62    5        16 != x_130
     

                    16000        140    1         0 != y_12
                    16000        140    2         0 != x_119
                    16000        144    3   F     3  = x_50
                    16000        166    4         4 != pr_86
                    16000        144    5   F     3  = x_50
 ! Time = 103.87s, Memory usage = 2.9 GB
 ! Current bound is 0
 !          Best Branches  Non-fixed    W       Branch decision
                    15000        198    6        34  = x_48
                    15000        259    7         0 != pr_41
                    12000        184    8   F     3 != x_83
                    17000        322    1   F     2  = y_71
                    17000        144    3   F    15  = x_139
                    17000        143    4         0 != pr_76
                    17000        144    5   F    15  = x_139
                    16000        103    7        28 != x_51
                    13000         59    8         0  = y_60
                    18000  

                    31000        184    4         9  = x_27
 ! Time = 159.03s, Memory usage = 2.9 GB
 ! Current bound is 0
 !          Best Branches  Non-fixed    W       Branch decision
                    30000        142    5   F     2  = y_88
                    30000         55    6   F        -
                    29000        335    7         0  = x_59
                    27000        321    8         0  = y_73
                    34000        141    1         6  = x_66
                    34000        255    2        10  = x_23
                    32000        113    4   F    10  = x_141
                    30000        117    7   F     3 != y_55
                    35000         47    1   F    30  = x_49
                    31000        145    3   F    14  = x_118
                    31000        145    5   F    14  = x_118
                    31000        145    6         7 != x_55
                    31000        117    7   F     7 != x_94
                    36000         5

                    45000        404    6   F    34  = x_121
                    48000        321    7         0  = x_60
                    44000        158    3         1  = pr_129
                    44000        158    5         1  = pr_129
                    46000        408    6        31 != x_2
                    44000        283    8   F     3  = x_145
                    45000        195    3        24 != z_142
                    45000        195    5        24 != z_142
                    47000        303    6   F     0  = x_36
                    45000        287    8   F    32  = x_4
                    46000        316    1        45 != x_125
                    49000        359    2   F     2  = y_133
                    46000        200    3   F        -
                    48000        225    4         0 != x_1
                    46000        200    5   F        -
                    48000        302    6   F     1 != x_126
                    49000        316    7 

                    63000        111    7         0  = pr_82
                    60000        143    1   F     3  = x_48
                    64000        321    2         1 != x_16
                    63000         56    3        79 != z_111
                    60000         99    4        49  = x_129
                    63000         56    5        79 != z_111
                    64000        109    7         5  = x_36
                    61000        140    1       130  = z_56
                    61000        100    4         5 != x_38
                    65000        180    7   F     7 != x_96
                    62000        184    1         0  = pt_48
                    65000        184    2   F     3 != x_83
                    64000         61    3   F     9  = x_37
                    62000        127    4   F    47 != x_47
                    64000         61    5   F     9  = x_37
 ! Time = 314.72s, Memory usage = 2.9 GB
 ! Current bound is 0
 !          Best Branches  Non-f

...

                     133k        306    4   F     3  = x_139
                     150k        371    8   F     3  = x_121
                     150k        300    7       143  = z_2
                     151k        371    8         1  = y_120
                     140k        238    1         2  = y_15
                     134k        335    2   F     6  = x_37
                     134k        306    4         4 != x_36
                     133k        295    6            -
                     152k        357    8         8 != x_38
                     141k        234    1         1  = y_62
                     135k        335    2   F     0 != y_92
                     135k        395    3         5 != x_95
                     135k        307    4         7  = x_139
                     135k        395    5         5 != x_95
                     134k        184    6            -
                     151k        295    7         4 != x_36
                     153k        357    8   F  

             439     149k        180    5         5  = x_87
             439     148k         69    6         4 != pr_8
             439     162k        185    7   F     1  = y_92
             439     169k        104    8   F     4  = pr_61
 *           438     156k  801.44s      1      (gap is 100.0%)
 *           437     156k  801.53s      1      (gap is 100.0%)
             437     157k          1    1         1  = x_79
 *           436     157k  801.59s      1      (gap is 100.0%)
             436     150k          5    2         5  = z_67
             436     150k        122    3   F     0 != pr_120
             436     146k        299    4        38 != x_10
             436     150k        122    5   F     0 != pr_120
             436     149k         71    6   F     3  = x_144
             436     163k         90    7         0  = x_54
             436     170k        103    8         4  = z_127
 ! Time = 804.44s, Average fail depth = 48, Memory usage = 2.9 GB
 ! Current bound i

             118     163k          4    2   F     1  = x_72
             118     160k        150    6   F     4  = y_63
             118     174k        305    7   F     1  = y_139
             118     181k        153    8         3  = z_127
             118     172k          4    1   F     0 != x_13
             118     164k          1    2       139  = z_93
             118     160k          1    4         7  = x_95
             118     162k        220    5   F     6 != x_95
             118     161k        153    6         5 != x_54
             118     175k        319    7   F     1 != y_90
             118     182k         98    8        27  = x_41
             118     173k          3    1        22  = x_8
             118     165k          1    2        22  = x_43
             118     162k        220    3   F     6 != x_95
             118     161k          6    4         2  = x_3
             118     163k        140    5         0  = pr_6
 ! Time = 855.02s, Average fail depth = 

             103     180k          6    2         0  = pr_15
             103     177k        105    3   F     0 != pr_44
             103     177k        105    5   F     0 != pr_44
             103     174k        104    6        38  = z_96
             103     184k        303    7   F     5  = x_94
             103     195k        301    8   F     0  = x_88
             103     188k          5    1   F    23 != x_69
             103     176k          1    4   F    43  = x_9
             103     178k        111    5        26 != x_121
             103     175k        182    6   F     2  = x_83
             103     185k        303    7         1 != x_0
             103     196k        304    8   F     0  = x_96
             103     189k         41    1         2  = x_103
             103     181k         99    2   F     3  = x_74
             103     178k        111    3        26 != x_121
             103     177k         37    4        59  = z_134
 ! Time = 912.34s, Average fail dep

             103     196k         80    2   F     3 != x_132
             103     189k        180    3        11 != x_144
             103     192k         75    4         9  = x_145
 *            23     204k  974.89s      1      (gap is 100.0%)
              23     197k         30    2   F     2  = x_6
              23     190k         96    3         4  = x_82
              23     193k         50    4   F     6 != x_129
              23     190k         96    5         4  = x_82
              23     190k         93    6   F        -
              23     201k         98    7   F     2  = x_132
              23     208k        208    8   F    11  = x_42
              23     205k          1    1         4  = x_6
              23     198k         30    2   F     3 != x_9
              23     191k        184    3            -
              23     194k         18    4        21 != x_8
              23     191k        184    5            -
 ! Time = 980.04s, Average fail depth = 93, Memory 

In [14]:
sol.print_solution()

-------------------------------------------------------------------------------
Model constraints: 606300, variables: integer: 799, interval: 0, sequence: 0
Solve status: Feasible
Search status: SearchStopped, stop cause: SearchStoppedByLimit
Solve time: 1001.38 sec
-------------------------------------------------------------------------------
Objective values: (23,), bounds: (0,), gaps: (1,)
Variables:
   pr_0 = 0
   pr_1 = 0
   pr_2 = 0
   pr_3 = 0
   pr_4 = 0
   pr_5 = 0
   pr_6 = 0
   pr_7 = 0
   pr_8 = 0
   pr_9 = 0
   pr_10 = 1
   pr_11 = 0
   pr_12 = 0
   pr_13 = 0
   pr_14 = 1
   pr_15 = 1
   pr_16 = 0
   pr_17 = 0
   pr_18 = 0
   pr_19 = 0
   pr_20 = 0
   pr_21 = 0
   pr_22 = 0
   pr_23 = 0
   pr_24 = 0
   pr_25 = 0
   pr_26 = 0
   pr_27 = 0
   pr_28 = 0
   pr_29 = 0
   pr_30 = 0
   pr_31 = 0
   pr_32 = 0
   pr_33 = 0
   pr_34 = 0
   pr_35 = 0
   pr_36 = 0
   pr_37 = 0
   pr_38 = 0
   pr_39 = 0
   pr_40 = 0
   pr_41 = 0
   pr_42 = 0
   pr_43 = 0
   pr_44 = 0
   pr_45 = 0
   p

In [15]:
check_solution(sol)

---no output if all constraints passed---


In [16]:
print('Objective: ', sol.get_objective_value())
print('Total runtime: ', sol.get_solve_time(), 'seconds')

Objective:  23
Total runtime:  1001.38 seconds
